thank you TAREK HAMDI https://www.kaggle.com/code/hamditarek/similar-image-cnn-cosine-similarity
this is amazing notebook and thank you for your knowledge sharing! I will inspire to do the same and contribute to the community.

# 1. Importing Libraries

In [ ]:
pip install --upgrade pip

In [11]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer
from keras.applications.xception import Xception,preprocess_input
import tensorflow as tf
from keras.preprocessing import image
from keras.layers import Input
from keras.backend import reshape
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

# 2. Loading and Preprocessing of Data

In [13]:
os.getcwd()


'C:\\Users\\19095\\Documents\\Capstone\\CapstoneH-M'

In [15]:
# images_dir = '../input/h-and-m-personalized-fashion-recommendations/images'
images_dir = 'C:\\Users\\19095\\Documents\\Capstone\\Dataset\\HandM\\images'

In [17]:
images_path = getImagePaths(images_dir)

In [18]:
len(images_path)

105100

In [19]:
X = []

In [20]:
for path in images_path[:8000]:
    X.append(preprocess_img(path))

In [21]:
# each X pixels should be scaled 
X = np.array(X)

In [22]:
X.shape

(8000, 225, 225, 3)

In [28]:
string_article_id = []

for path in images_path[:8000]:
#     string_article_id.append(int(path.split('/')[5][:10]))
    string_article_id.append(int(path.split('\\')[9][:10]))

In [30]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

articles = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")
cols = ['article_id','product_group_name']
temp = articles[cols]

label = pd.DataFrame(string_article_id, columns=['article_id'])
label = label.merge(temp, how = 'left', left_on='article_id',right_on='article_id')


label['product_group_name_coded'] = le.fit_transform(label['product_group_name'])

In [31]:
len(string_article_id)

8000

In [32]:
label.shape

(8000, 3)

In [ ]:
X.shape

In [33]:
from sklearn.model_selection import train_test_split
y = label['product_group_name_coded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [34]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(225, 225, 3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(19)
])

In [35]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [36]:
model.fit(X_train, y_train, epochs=30)

Epoch 1/30
168/168 [==============================] - 9s 52ms/step - loss: 18.3557 - accuracy: 0.4084
Epoch 2/30
168/168 [==============================] - 9s 53ms/step - loss: 4.2125 - accuracy: 0.3933
Epoch 3/30
168/168 [==============================] - 9s 53ms/step - loss: 3.6936 - accuracy: 0.4916
Epoch 4/30
168/168 [==============================] - 9s 56ms/step - loss: 3.7643 - accuracy: 0.5075
Epoch 5/30
168/168 [==============================] - 9s 55ms/step - loss: 3.4808 - accuracy: 0.4815
Epoch 6/30
168/168 [==============================] - 9s 54ms/step - loss: 2.5781 - accuracy: 0.5052
Epoch 7/30
168/168 [==============================] - 9s 54ms/step - loss: 2.6295 - accuracy: 0.5200
Epoch 8/30
168/168 [==============================] - 9s 53ms/step - loss: 3.4565 - accuracy: 0.5235
Epoch 9/30
168/168 [==============================] - 9s 53ms/step - loss: 2.1781 - accuracy: 0.5410
Epoch 10/30
168/168 [==============================] - 9s 53ms/step - loss: 2.3768 - accur

In [37]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

83/83 - 3s - loss: 5.5405 - accuracy: 0.5364 - 3s/epoch - 31ms/step

Test accuracy: 0.5363636612892151


In [41]:
np.expand_dims(X_test[0],axis=0).shape

(1, 225, 225, 3)

In [42]:
model.predict(np.expand_dims(X_test[0],axis=0))

array([[  24.819036, -314.50995 ,   23.735468,   62.529076,   16.321854,
         -25.804443,   49.197674,  -32.20737 ,   51.375164,   50.49598 ,
          48.388817,  -47.81683 ,  -39.936295, -467.46664 ,  -97.01742 ,
        -133.55089 , -437.42303 , -320.12933 , -113.06713 ]],
      dtype=float32)

In [44]:
np.argmax(model.predict(np.expand_dims(X_test[0],axis=0)))

3

In [45]:
y_test[0]

4

In [16]:
def getImagePaths(path):
    """
    Function to Combine Directory Path with individual Image Paths
    
    parameters: path(string) - Path of directory
    returns: image_names(string) - Full Image Path
    """
    image_names = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            fullpath = os.path.join(dirname, filename)
            image_names.append(fullpath)
    return image_names

def preprocess_img(img_path):
    dsize = (225,225)
    new_image=cv2.imread(img_path)
    new_image=cv2.resize(new_image,dsize,interpolation=cv2.INTER_NEAREST)  
    
    # Add the image to a batch where it's the only member.
#     new_image=np.expand_dims(new_image,axis=0)
    
    # preprocess_input on your inputs before passing them to the model. xception.preprocess_input will scale input pixels between -1 and 1.
    new_image=preprocess_input(new_image)
    
    return new_image

def load_data():
    # return 10000 images
    output=[]
    output=getImagePaths(images_dir)[:10000]
    
    return output

# 3. Defining model and extracting feature for all the training data

In [ ]:
def model():
#     model=Xception(weights='imagenet',include_top=False)
#     for layer in model.layers:
#         layer.trainable=False
#         #model.summary()
#     return model

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(225, 225, 3)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

def feature_extraction(image_data,model):
    features=model.predict(image_data)
    features=np.array(features)
    features=features.flatten()
    return features

# 4. Finding the similar image through LSH and cosine similarity

In [ ]:
def result_vector_cosine(model,feature_vector,new_img):
    new_feature = model.predict(new_img)
    new_feature = np.array(new_feature)
    new_feature = new_feature.flatten()
    N_result = 12
    nbrs = NearestNeighbors(n_neighbors=N_result, metric="cosine").fit(feature_vector)
    distances, indices = nbrs.kneighbors([new_feature])
    return(indices)

In [ ]:
main_model=model()

# 5. Result
## Vector Cosine

In [ ]:
def input_show(data):
    plt.title("Query Image")
    plt.imshow(data)
  
def show_result(data,result):
    fig = plt.figure(figsize=(12,8))
    for i in range(0,12):
        index_result=result[0][i]
        plt.subplot(3,4,i+1)
        plt.imshow(cv2.imread(data[index_result]))
    plt.show()

def main():  
    features=[]
    output=load_data()
    main_model=model()
    #Limiting the data for training
    for i in output[:999]:
        new_img=preprocess_img(i)
        features.append(feature_extraction(new_img,main_model))
    feature_vec = np.array(features)
    result=result_vector_cosine(main_model,feature_vec,preprocess_img(output[1000]))
    input_show(cv2.imread(output[1000]))
    show_result(output,result)
  

if __name__=='__main__':
    main()

In [ ]:
main()